# Gmail Module — Test Notebook

Test the Gmail pipeline components: email parsing, reply generation, and sending.

Sections 1-3 run without API credentials. Section 4 requires Gmail OAuth2 setup.

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "gmail" else Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

## 1. Email Parser

Test parsing raw Gmail API message dicts into friendly formats.

In [ ]:
from gmail.utils.parser import parse_message, get_header
import json
import base64

# Simulated Gmail API message (format=full)
body_text = "Hi there,\n\nI'd like to know about your return policy.\n\nThanks,\nAlice"
encoded_body = base64.urlsafe_b64encode(body_text.encode("utf-8")).decode("utf-8")

raw_message = {
    "id": "msg-abc123",
    "threadId": "thread-xyz789",
    "labelIds": ["INBOX", "UNREAD"],
    "snippet": "Hi there, I'd like to know about your return policy.",
    "payload": {
        "mimeType": "text/plain",
        "headers": [
            {"name": "Subject", "value": "Return Policy Question"},
            {"name": "From", "value": "Alice <alice@example.com>"},
            {"name": "To", "value": "shop@jadedrose.com"},
            {"name": "Date", "value": "Mon, 3 Feb 2025 10:30:00 -0500"},
        ],
        "body": {"data": encoded_body},
    },
}

parsed = parse_message(raw_message)
print("Parsed email:")
print(json.dumps(parsed, indent=2, default=str))

In [ ]:
# Test get_header utility
headers = raw_message["payload"]["headers"]
print(f"Subject: {get_header(headers, 'Subject')}")
print(f"From:    {get_header(headers, 'From')}")
print(f"Missing: '{get_header(headers, 'X-Custom')}'")

## 2. Generate Replies (single email)

The `run()` function takes email body text and returns a reply.

In [ ]:
from gmail.handlers.build_replies import run, generate_reply

# Single email reply (default: echoes back)
email_body = "Hi, I'd like to return an item I purchased last week."
subject = "Return Request"

reply = run(email_body, subject=subject)
print(f"Email body: {email_body}")
print(f"Subject:    {subject}")
print(f"Reply:      {reply}")

## 3. Build Replies from Trigger Entries

Process a batch of email trigger entries.

In [ ]:
from gmail.handlers.build_replies import build_replies

# Example trigger entries (simulating emails from Gmail)
email_triggers = [
    {
        "id": "trigger-001",
        "gmail_message_id": "msg-abc123",
        "gmail_thread_id": "thread-xyz789",
        "from": "alice@example.com",
        "subject": "Return Policy Question",
        "message": {
            "text": "What is your return policy?",
            "snippet": "What is your return policy?",
        },
    },
    {
        "id": "trigger-002",
        "gmail_message_id": "msg-def456",
        "gmail_thread_id": "thread-uvw321",
        "from": "bob@example.com",
        "subject": "Shipping Question",
        "message": {
            "text": "Do you ship to Canada?",
            "snippet": "Do you ship to Canada?",
        },
    },
    {
        "id": "trigger-003",
        "gmail_message_id": "msg-ghi789",
        "gmail_thread_id": "thread-rst654",
        "from": "alice@example.com",
        "subject": "Order Status",
        "message": {
            "text": "Where is my order #5678?",
            "snippet": "Where is my order #5678?",
        },
    },
]

replies = build_replies(email_triggers)
print(f"Built {len(replies)} replies:\n")
for r in replies:
    print(f"  To: {r['to']}")
    print(f"  Subject: {r['subject']}")
    print(f"  Reply: {r['reply']['text']}")
    print()

In [ ]:
# Filter by sender
alice_replies = build_replies(email_triggers, filter_from="alice@example.com")
print(f"Alice's replies: {len(alice_replies)}")
for r in alice_replies:
    print(f"  -> {r['subject']}: {r['reply']['text']}")

## 4. Send Replies via Gmail (requires OAuth2 credentials)

This sends real emails via the Gmail API.
Set up OAuth2 credentials before running.

In [ ]:
from gmail.actions.send_replies import send_all

# Example reply entries to send
reply_entries = [
    {
        "to": "test@example.com",
        "subject": "Test from notebook",
        "reply": {"text": "This is a test reply from the Gmail test notebook."},
    },
]

# Uncomment and run when OAuth2 is configured:
# from gmail.utils.auth import get_gmail_service
# service = get_gmail_service()
# sent = send_all(service, reply_entries)
# print(f"Sent {sent} email(s)")

print("Uncomment the lines above after setting up Gmail OAuth2 credentials.")

## 5. Poll Inbox (requires OAuth2 credentials)

Fetch unread emails directly as a list.

In [ ]:
from gmail.triggers.poll_inbox import poll_inbox

# Uncomment and run when OAuth2 is configured:
# from gmail.utils.auth import get_gmail_service
# service = get_gmail_service()
# entries = poll_inbox(service, max_results=5, mark_as_read=False)
# print(f"Found {len(entries)} unread email(s)")
# for e in entries:
#     print(f"  From: {e['from']}  Subject: {e['subject']}")

print("Uncomment the lines above after setting up Gmail OAuth2 credentials.")